In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan2 import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# !TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [8]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

input_dir_val = 'data/val/low/'
tgt_dir_val = 'data/val/high/'


test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'models/vgg16_notop.hdf5'

input_shape = (270, 480, 3)
tgt_shape = (540, 960, 3)

batch_size = 2

In [9]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
# test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
# test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

# test_img_sr = srgan.generator.predict(test_img_low)
# test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
# cv2.imwrite(test_test_path, test_img_sr_int)

In [11]:
for e in range(10001):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:1.1484068632125854, d_loss_fake:0.38119134306907654, vgg_loss:17.1591854095459
epoch 100
d_loss_real:0.7196857929229736, d_loss_fake:0.6614207029342651, vgg_loss:3.5309343338012695
epoch 200
d_loss_real:0.7170820236206055, d_loss_fake:0.6391987800598145, vgg_loss:2.3811795711517334
epoch 300
d_loss_real:0.6957992315292358, d_loss_fake:0.6954551935195923, vgg_loss:2.9955098628997803
epoch 400
d_loss_real:0.697943925857544, d_loss_fake:0.6946111917495728, vgg_loss:3.131446599960327
epoch 500
d_loss_real:0.7416832447052002, d_loss_fake:0.7165399789810181, vgg_loss:2.930420160293579
epoch 600
d_loss_real:0.6703459024429321, d_loss_fake:0.6869553327560425, vgg_loss:2.786417007446289
epoch 700
d_loss_real:0.7435316443443298, d_loss_fake:0.7065401077270508, vgg_loss:1.951074242591858
epoch 800
d_loss_real:0.7239294052124023, d_loss_fake:0.6618557572364807, vgg_loss:1.2817351818084717
epoch 900
d_loss_real:0.7529546618461609, d_loss_fake:0.6776461005210876, vgg_loss:1.60471

d_loss_real:0.6833106875419617, d_loss_fake:0.6442772746086121, vgg_loss:1.0728516578674316
epoch 8100
d_loss_real:0.7413156628608704, d_loss_fake:0.7254693508148193, vgg_loss:0.7106648087501526
epoch 8200
d_loss_real:0.7031893730163574, d_loss_fake:0.6904920339584351, vgg_loss:0.7868041396141052
epoch 8300
d_loss_real:0.7178293466567993, d_loss_fake:0.6323229074478149, vgg_loss:0.8455759286880493
epoch 8400
d_loss_real:0.7088638544082642, d_loss_fake:0.7378358840942383, vgg_loss:1.6042277812957764
epoch 8500
d_loss_real:0.7142859697341919, d_loss_fake:0.6675015687942505, vgg_loss:0.8343786597251892
epoch 8600
d_loss_real:0.7022349238395691, d_loss_fake:0.6752859354019165, vgg_loss:1.4925854206085205
epoch 8700
d_loss_real:0.7090908288955688, d_loss_fake:0.6545664072036743, vgg_loss:0.7444463968276978
epoch 8800
d_loss_real:0.6938109397888184, d_loss_fake:0.6627925634384155, vgg_loss:1.5483609437942505
epoch 8900
d_loss_real:0.7126727104187012, d_loss_fake:0.630710244178772, vgg_loss:1

In [12]:
srgan.generator.save('models/20200512_generator.h5', include_optimizer=False)

In [13]:
from keras.models import Model, load_model
from keras.engine.network import Network
from keras.layers import Input

# pre_model = load_model('20200510_generator.h5')
pre_model = srgan.generator
pre_net = Network(inputs=pre_model.input, outputs=pre_model.get_layer(pre_model.layers[-1].name).output, name='learned')

input_layer = Input(shape = (540, 960, 3))
sr_out = pre_net(input_layer)
sr_generator = Model(input_layer, sr_out)


In [14]:
img_list = []
for i in range(4):
    test_file = f'UBMk30rjy0o_17675_{i}.jpg'
    path = input_dir_val + test_file
    test_img_low = ((cv2.imread(path) - 127.5) / 127.5)
    img_list.append(test_img_low)

h0 = np.vstack([img_list[0], img_list[1]])
h1 = np.vstack([img_list[2], img_list[3]])
img = np.hstack([h0, h1])
img_gen = sr_generator.predict(img.reshape((1, 540, 960, 3)))

img_int = ((img_gen * 127.5) + 127.5).astype('u1').reshape((1080, 1920, 3))
cv2.imwrite('data/test/20200511_out_premerged.jpg', img_int)

True